In [1]:
#dependiencies
import pandas as pd
from sodapy import Socrata
from key import token,username,password

In [2]:
#client to connect to cdc covid-19 data
client = Socrata("data.cdc.gov",
                  token,
                  username,
                  password)

In [3]:
# results from cdc vbim-akqf
results = client.get("9mfq-cb36", limit=23000)

In [4]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-05-04T00:00:00.000,KY,5245,5245.0,0,115.0,0,261,261.0,0,8.0,0,2020-05-05T17:25:08.000,Agree,Agree
1,2020-12-08T00:00:00.000,OK,205999,171497.0,34502,0.0,0,1752,1680.0,72,0.0,0,2020-12-09T14:45:40.234,Agree,Agree
2,2020-08-21T00:00:00.000,MD,103523,NaN,NaN,624.0,0,3685,3546.0,139,11.0,1,2020-08-22T14:21:09.869,N/A,Agree
3,2020-04-01T00:00:00.000,MN,742,NaN,NaN,53.0,NaN,18,NaN,NaN,1.0,NaN,2020-03-31T16:22:39.452,N/A,Agree
4,2020-06-25T00:00:00.000,NE,18346,NaN,NaN,125.0,0,260,NaN,NaN,3.0,0,2020-06-26T19:18:27.809,Not agree,Not agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22135,2020-10-28T00:00:00.000,OR,43228,41508.0,1720,420.0,0,671,656.0,15,7.0,0,2020-10-29T14:11:33.577,Agree,Agree
22136,2020-09-06T00:00:00.000,OR,28044,26737.0,1307,188.0,0,481,469.0,12,1.0,0,2020-09-07T13:37:36.613,Agree,Agree
22137,2020-10-19T00:00:00.000,KS,72968,67935.0,5033,2113.0,179,872,NaN,NaN,13.0,0,2020-10-20T14:59:27.967,Agree,N/A
22138,2020-02-26T00:00:00.000,KS,0,NaN,NaN,0.0,NaN,0,NaN,NaN,0.0,NaN,2020-03-26T16:22:39.452,Agree,N/A


In [6]:
#reformatting date
results_df['submission_date']=pd.to_datetime(results_df['submission_date']).dt.date
results_df['created_at']=pd.to_datetime(results_df['created_at']).dt.date

In [7]:
results_df

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-05-04,KY,5245,5245.0,0,115.0,0,261,261.0,0,8.0,0,2020-05-05,Agree,Agree
1,2020-12-08,OK,205999,171497.0,34502,0.0,0,1752,1680.0,72,0.0,0,2020-12-09,Agree,Agree
2,2020-08-21,MD,103523,NaN,NaN,624.0,0,3685,3546.0,139,11.0,1,2020-08-22,N/A,Agree
3,2020-04-01,MN,742,NaN,NaN,53.0,NaN,18,NaN,NaN,1.0,NaN,2020-03-31,N/A,Agree
4,2020-06-25,NE,18346,NaN,NaN,125.0,0,260,NaN,NaN,3.0,0,2020-06-26,Not agree,Not agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22135,2020-10-28,OR,43228,41508.0,1720,420.0,0,671,656.0,15,7.0,0,2020-10-29,Agree,Agree
22136,2020-09-06,OR,28044,26737.0,1307,188.0,0,481,469.0,12,1.0,0,2020-09-07,Agree,Agree
22137,2020-10-19,KS,72968,67935.0,5033,2113.0,179,872,NaN,NaN,13.0,0,2020-10-20,Agree,N/A
22138,2020-02-26,KS,0,NaN,NaN,0.0,NaN,0,NaN,NaN,0.0,NaN,2020-03-26,Agree,N/A


In [8]:
from sqlalchemy import create_engine

In [9]:
from config import location

In [10]:
engine=create_engine(location)

In [11]:
from sqlalchemy import event

In [12]:
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_excute(
    conn, cursor, statement, params, context, excutemany
    ):
        if excutemany:
            cursor.fast_excutemany=True

In [13]:
receive_before_cursor_excute

<function __main__.receive_before_cursor_excute(conn, cursor, statement, params, context, excutemany)>

In [14]:
#pandas dataframe stored in a postgresql database
results_df.to_sql('covid_19_us_cases',engine,index=False,if_exists="replace",chunksize=100,method='multi')